In [1]:
import os
import re

import shutil
import codecs, json 
import nibabel as nib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc
from scipy import stats
import sklearn as sk
import time
from dypac import Dypac
import nilearn
from nilearn import image, plotting, datasets
from nilearn.input_data import NiftiLabelsMasker, NiftiMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn.decomposition import DictLearning, CanICA
from scipy.stats import pearsonr
import nilearn.plotting as plotting
import csv

import hcp_utils as hcp
import conf

import mne
from mne.datasets import sample
from mne.minimum_norm import apply_inverse_epochs, read_inverse_operator
from mne.connectivity import spectral_connectivity
from mne.viz import circular_layout, plot_connectivity_circle
import plot_con_helper
import feature_selec_functions
import graph_comparison_helper
import importlib

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


### Load data

All features

In [2]:
all_fc_data = pd.read_csv("all_fc_data.csv")
all_fc_data = all_fc_data.loc[:, ~all_fc_data.columns.str.contains('^Unnamed')] #delete un-named column

In [3]:
importlib.reload(graph_comparison_helper)
all_features_temp = all_fc_data.columns
#convert from strings to tuples
all_features = graph_comparison_helper.conv_to_tuple(all_features_temp)

#all_features

Models' features

In [4]:
importlib.reload(plot_con_helper)

#Paths to results 
PATH = conf.RESULTS_DIR

#Plot results of all 3 tasks
#dfs = ['df_DD_l1FixedFisher_02', 'df_F_l1FixedFisher_02', 'df_PicVocab_Unil1FixedFisher_02']

dfs = ['DDisc_AUC_40K', 'Flanker_AgeAdj']

df_arranged = {}
for i in range(len(dfs)):
    path = PATH + '{}.csv'.format(dfs[i])
    df = pd.read_csv(path)
    #df = df[['node1', 'node2', 'R']] #For reg analysis
    df = df[['node1', 'node2', 'beta', 'rel_edges']] #For models
    df.columns = ['node1', 'node2', 'beta', 'rel_edges']
    df_arranged[dfs[i]] = df
    

Parcels list

In [5]:
#path to parcellation labels 
path_label = '/mnt/c/Users/liz/Contacts/Desktop/ValuePercept/Parcellations/Parcellations/HCP/fslr32k/cifti/Schaefer2018_100Parcels_7Networks_order_info.txt'

with open(path_label, 'r') as f:
    lines = f.readlines()

In [6]:
#list of parcels
list_parcel = []
for i in range(0, len(lines), 2): #only even rows
    list_parcel.append(lines[i][10:-1])
#len(list_parcel)
#list_parcel

Create dfs for comparison

In [7]:
#selected features of each model
DD_features_temp = list(df_arranged['DDisc_AUC_40K'].rel_edges)
F_features_temp = list(df_arranged['Flanker_AgeAdj'].rel_edges)
#convert from strings to tuples
DD_features = graph_comparison_helper.conv_to_tuple(DD_features_temp)
F_features = graph_comparison_helper.conv_to_tuple(F_features_temp)

In [8]:
#betas of each model
DD_beta = list(df_arranged['DDisc_AUC_40K'].beta)
F_beta = list(df_arranged['Flanker_AgeAdj'].beta)

In [9]:
importlib.reload(graph_comparison_helper)
df_DD = graph_comparison_helper.arrange_df(all_features, DD_features, DD_beta)
df_F = graph_comparison_helper.arrange_df(all_features, F_features, F_beta)

In [10]:
#validation
df_F.loc[df_F['all_features'] == (1,26)]

,all_features,model_features_logic,beta
123,"(1, 26)",1,0.43949


In [11]:
df_F.index

RangeIndex(start=0, stop=4950, step=1)

Add parcel names

In [12]:
importlib.reload(graph_comparison_helper)
node_names = graph_comparison_helper.get_node_names(all_features, list_parcel)
df_DD = graph_comparison_helper.add_node_names(df_DD, node_names)
df_F = graph_comparison_helper.add_node_names(df_F, node_names)

Add net names

In [13]:
plot_con_helper.add_col(df_DD)
plot_con_helper.add_col(df_F)

LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_1
LH_Vis_2
LH_Vis_2
LH_Vis_2
LH_Vis_2
LH_Vis_2
LH_Vis_2
LH_Vis_2
LH_Vis_2
LH_Vis_2
LH_Vis_2
LH_Vis_2
LH_Vis_2
L

LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_Post_4
LH_DorsAttn_

LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_1
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par_2
LH_Default_Par

RH_Cont_PFCl_2
RH_Cont_PFCl_2
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_3
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_PFCl_4
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_Cing_1
RH_Cont_PFCmp_1
RH_Cont_PFCmp_1
RH_Cont_PFCmp_1
RH_Cont_PFCmp_1
RH_Cont_PFCmp_1
RH_Cont_PFCmp_1
RH_Cont_PFCmp_1
RH_Cont_PFCmp_1
RH_Cont_PFCmp_1
RH_Cont_PFCmp_1
RH_Cont_PFCmp_1
RH_Cont_PFCmp_1
RH_Cont_pCun_1
RH_Cont_pCun_1
RH_Cont_pCun_1
RH_Cont_pCun_1
RH_Cont_pCun_1
RH_Cont_pCun_1
RH_Cont_pCun_1
RH_Cont_pCun_1
RH_Cont_pCun_1
RH_Cont_pCun_

RH_Default_PFCdPFCm_2
RH_Default_PFCdPFCm_3
RH_Default_pCunPCC_1
RH_Default_pCunPCC_2
LH_DorsAttn_Post_3
LH_DorsAttn_Post_4
LH_DorsAttn_Post_5
LH_DorsAttn_Post_6
LH_DorsAttn_PrCv_1
LH_DorsAttn_FEF_1
LH_SalVentAttn_ParOper_1
LH_SalVentAttn_FrOperIns_1
LH_SalVentAttn_FrOperIns_2
LH_SalVentAttn_PFCl_1
LH_SalVentAttn_Med_1
LH_SalVentAttn_Med_2
LH_SalVentAttn_Med_3
LH_Limbic_OFC_1
LH_Limbic_TempPole_1
LH_Limbic_TempPole_2
LH_Cont_Par_1
LH_Cont_PFCl_1
LH_Cont_pCun_1
LH_Cont_Cing_1
LH_Default_Temp_1
LH_Default_Temp_2
LH_Default_Par_1
LH_Default_Par_2
LH_Default_PFC_1
LH_Default_PFC_2
LH_Default_PFC_3
LH_Default_PFC_4
LH_Default_PFC_5
LH_Default_PFC_6
LH_Default_PFC_7
LH_Default_pCunPCC_1
LH_Default_pCunPCC_2
RH_Vis_1
RH_Vis_2
RH_Vis_3
RH_Vis_4
RH_Vis_5
RH_Vis_6
RH_Vis_7
RH_Vis_8
RH_SomMot_1
RH_SomMot_2
RH_SomMot_3
RH_SomMot_4
RH_SomMot_5
RH_SomMot_6
RH_SomMot_7
RH_SomMot_8
RH_DorsAttn_Post_1
RH_DorsAttn_Post_2
RH_DorsAttn_Post_3
RH_DorsAttn_Post_4
RH_DorsAttn_Post_5
RH_DorsAttn_PrCv_1
RH_Dors

...

RH_DorsAttn_FEF_1
RH_DorsAttn_FEF_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_2
RH_SalVentAttn_TempOccPar_2
RH_SalVentAttn_TempOccPar_2
RH_SalVentAttn_TempOccPar_2
RH_SalVentAttn_TempOccPar_2
RH_SalVentAttn_TempOccPar_2
RH_SalVentAttn_TempOccPar_2
RH_SalVentAttn_TempOccPar_2
RH_SalVentAt

RH_SomMot_5
RH_SomMot_6
RH_SomMot_7
RH_SomMot_8
RH_DorsAttn_Post_1
RH_DorsAttn_Post_2
RH_DorsAttn_Post_3
RH_DorsAttn_Post_4
RH_DorsAttn_Post_5
RH_DorsAttn_PrCv_1
RH_DorsAttn_FEF_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_2
RH_SalVentAttn_FrOperIns_1
RH_SalVentAttn_Med_1
RH_SalVentAttn_Med_2
RH_Limbic_OFC_1
RH_Limbic_TempPole_1
RH_Cont_Par_1
RH_Cont_Par_2
RH_Cont_PFCl_1
RH_Cont_PFCl_2
RH_Cont_PFCl_3
RH_Cont_PFCl_4
RH_Cont_Cing_1
RH_Cont_PFCmp_1
RH_Cont_pCun_1
RH_Default_Par_1
RH_Default_Temp_1
RH_Default_Temp_2
RH_Default_Temp_3
RH_Default_PFCv_1
RH_Default_PFCv_2
RH_Default_PFCdPFCm_1
RH_Default_PFCdPFCm_2
RH_Default_PFCdPFCm_3
RH_Default_pCunPCC_1
RH_Default_pCunPCC_2
LH_DorsAttn_Post_1
LH_DorsAttn_Post_2
LH_DorsAttn_Post_3
LH_DorsAttn_Post_4
LH_DorsAttn_Post_5
LH_DorsAttn_Post_6
LH_DorsAttn_PrCv_1
LH_DorsAttn_FEF_1
LH_SalVentAttn_ParOper_1
LH_SalVentAttn_FrOperIns_1
LH_SalVentAttn_FrOperIns_2
LH_SalVentAttn_PFCl_1
LH_SalVentAttn_Med_1
LH_SalVentAttn_Med_2
LH_SalVentAttn_

RH_SomMot_3
RH_SomMot_4
RH_SomMot_5
RH_SomMot_6
RH_SomMot_7
RH_SomMot_8
RH_DorsAttn_Post_1
RH_DorsAttn_Post_2
RH_DorsAttn_Post_3
RH_DorsAttn_Post_4
RH_DorsAttn_Post_5
RH_DorsAttn_PrCv_1
RH_DorsAttn_FEF_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_2
RH_SalVentAttn_FrOperIns_1
RH_SalVentAttn_Med_1
RH_SalVentAttn_Med_2
RH_Limbic_OFC_1
RH_Limbic_TempPole_1
RH_Cont_Par_1
RH_Cont_Par_2
RH_Cont_PFCl_1
RH_Cont_PFCl_2
RH_Cont_PFCl_3
RH_Cont_PFCl_4
RH_Cont_Cing_1
RH_Cont_PFCmp_1
RH_Cont_pCun_1
RH_Default_Par_1
RH_Default_Temp_1
RH_Default_Temp_2
RH_Default_Temp_3
RH_Default_PFCv_1
RH_Default_PFCv_2
RH_Default_PFCdPFCm_1
RH_Default_PFCdPFCm_2
RH_Default_PFCdPFCm_3
RH_Default_pCunPCC_1
RH_Default_pCunPCC_2
LH_Cont_Cing_1
LH_Default_Temp_1
LH_Default_Temp_2
LH_Default_Par_1
LH_Default_Par_2
LH_Default_PFC_1
LH_Default_PFC_2
LH_Default_PFC_3
LH_Default_PFC_4
LH_Default_PFC_5
LH_Default_PFC_6
LH_Default_PFC_7
LH_Default_pCunPCC_1
LH_Default_pCunPCC_2
RH_Vis_1
RH_Vis_2
RH_Vis_3
RH_Vis_4
RH_

RH_SalVentAttn_Med_2
RH_Limbic_OFC_1
RH_Limbic_TempPole_1
RH_Cont_Par_1
RH_Cont_Par_2
RH_Cont_PFCl_1
RH_Cont_PFCl_2
RH_Cont_PFCl_3
RH_Cont_PFCl_4
RH_Cont_Cing_1
RH_Cont_PFCmp_1
RH_Cont_pCun_1
RH_Default_Par_1
RH_Default_Temp_1
RH_Default_Temp_2
RH_Default_Temp_3
RH_Default_PFCv_1
RH_Default_PFCv_2
RH_Default_PFCdPFCm_1
RH_Default_PFCdPFCm_2
RH_Default_PFCdPFCm_3
RH_Default_pCunPCC_1
RH_Default_pCunPCC_2
RH_DorsAttn_Post_5
RH_DorsAttn_PrCv_1
RH_DorsAttn_FEF_1
RH_SalVentAttn_TempOccPar_1
RH_SalVentAttn_TempOccPar_2
RH_SalVentAttn_FrOperIns_1
RH_SalVentAttn_Med_1
RH_SalVentAttn_Med_2
RH_Limbic_OFC_1
RH_Limbic_TempPole_1
RH_Cont_Par_1
RH_Cont_Par_2
RH_Cont_PFCl_1
RH_Cont_PFCl_2
RH_Cont_PFCl_3
RH_Cont_PFCl_4
RH_Cont_Cing_1
RH_Cont_PFCmp_1
RH_Cont_pCun_1
RH_Default_Par_1
RH_Default_Temp_1
RH_Default_Temp_2
RH_Default_Temp_3
RH_Default_PFCv_1
RH_Default_PFCv_2
RH_Default_PFCdPFCm_1
RH_Default_PFCdPFCm_2
RH_Default_PFCdPFCm_3
RH_Default_pCunPCC_1
RH_Default_pCunPCC_2
RH_DorsAttn_PrCv_1
RH_Dors

,all_features,model_features_logic,beta,node1,node2,beta_abs,node1_nets,node2_nets
0,"(0, 1)",0,NaN,LH_Vis_1,LH_Vis_2,NaN,Vis,Vis
1,"(0, 2)",0,NaN,LH_Vis_1,LH_Vis_3,NaN,Vis,Vis
2,"(0, 3)",1,0.005773,LH_Vis_1,LH_Vis_4,0.005773,Vis,Vis
3,"(0, 4)",0,NaN,LH_Vis_1,LH_Vis_5,NaN,Vis,Vis
4,"(0, 5)",0,NaN,LH_Vis_1,LH_Vis_6,NaN,Vis,Vis
...,...,...,...,...,...,...,...,...
4945,"(96, 98)",0,NaN,RH_Default_PFCdPFCm_2,RH_Default_pCunPCC_1,NaN,Default,Default
4946,"(96, 99)",0,NaN,RH_Default_PFCdPFCm_2,RH_Default_pCunPCC_2,NaN,Default,Default
4947,"(97, 98)",0,NaN,RH_Default_PFCdPFCm_3,RH_Default_pCunPCC_1,NaN,Default,Default
4948,"(97, 99)",0,NaN,RH_Default_PFCdPFCm_3,RH_Default_pCunPCC_2,NaN,Default,Default


In [14]:
df_DD

,all_features,model_features_logic,beta,node1,node2,beta_abs,node1_nets,node2_nets
0,"(0, 1)",0,NaN,LH_Vis_1,LH_Vis_2,NaN,Vis,Vis
1,"(0, 2)",0,NaN,LH_Vis_1,LH_Vis_3,NaN,Vis,Vis
2,"(0, 3)",0,NaN,LH_Vis_1,LH_Vis_4,NaN,Vis,Vis
3,"(0, 4)",0,NaN,LH_Vis_1,LH_Vis_5,NaN,Vis,Vis
4,"(0, 5)",0,NaN,LH_Vis_1,LH_Vis_6,NaN,Vis,Vis
...,...,...,...,...,...,...,...,...
4945,"(96, 98)",0,NaN,RH_Default_PFCdPFCm_2,RH_Default_pCunPCC_1,NaN,Default,Default
4946,"(96, 99)",0,NaN,RH_Default_PFCdPFCm_2,RH_Default_pCunPCC_2,NaN,Default,Default
4947,"(97, 98)",0,NaN,RH_Default_PFCdPFCm_3,RH_Default_pCunPCC_1,NaN,Default,Default
4948,"(97, 99)",0,NaN,RH_Default_PFCdPFCm_3,RH_Default_pCunPCC_2,NaN,Default,Default


In [15]:
#validation
df_DD.loc[df_DD['all_features'] == (1,26)]

,all_features,model_features_logic,beta,node1,node2,beta_abs,node1_nets,node2_nets
123,"(1, 26)",1,0.015532,LH_Vis_2,LH_SalVentAttn_PFCl_1,0.015532,Vis,SalVentAttn


In [16]:
df_F

,all_features,model_features_logic,beta,node1,node2,beta_abs,node1_nets,node2_nets
0,"(0, 1)",0,NaN,LH_Vis_1,LH_Vis_2,NaN,Vis,Vis
1,"(0, 2)",0,NaN,LH_Vis_1,LH_Vis_3,NaN,Vis,Vis
2,"(0, 3)",1,0.005773,LH_Vis_1,LH_Vis_4,0.005773,Vis,Vis
3,"(0, 4)",0,NaN,LH_Vis_1,LH_Vis_5,NaN,Vis,Vis
4,"(0, 5)",0,NaN,LH_Vis_1,LH_Vis_6,NaN,Vis,Vis
...,...,...,...,...,...,...,...,...
4945,"(96, 98)",0,NaN,RH_Default_PFCdPFCm_2,RH_Default_pCunPCC_1,NaN,Default,Default
4946,"(96, 99)",0,NaN,RH_Default_PFCdPFCm_2,RH_Default_pCunPCC_2,NaN,Default,Default
4947,"(97, 98)",0,NaN,RH_Default_PFCdPFCm_3,RH_Default_pCunPCC_1,NaN,Default,Default
4948,"(97, 99)",0,NaN,RH_Default_PFCdPFCm_3,RH_Default_pCunPCC_2,NaN,Default,Default
